## Install Dependencies 

In [ ]:
!pip install -q git+https://github.com/THU-MIG/yolov10.git

In [ ]:
!pip install huggingface-hub

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

## Install pretrained models

In [ ]:
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!ls -lh {HOME}/weights

## Load Model

In [ ]:
from ultralytics import YOLOv10

In [ ]:
model = YOLOv10("/Users/alexis/Programmation/label-studio-ml-backend/yolo_backend/best.pt")

In [ ]:
model.train(data='dataset.yaml', epochs=500, batch=16, imgsz=640)

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/test/images/0a4299fb-frame_257.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/test/images/fafb4c4e-frame_40.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_8.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_72.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_116.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_323.jpg")

# Display the results
results[0].show()

In [ ]:
# Perform object detection on an image
results = model("/teamspace/studios/this_studio/data_test/frame_433.jpg")

# Display the results
results[0].show()

In [12]:

import cv2
import numpy as np
from ultralytics import YOLOv10

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train2_final/weights/best.pt")

# Perform object detection on an image
results = model("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/images/0a2d048d-frame_444.jpg")

# Display the results
results[0].boxes


image 1/1 /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/images/0a2d048d-frame_444.jpg: 480x640 1 fuel port, 305.8ms
Speed: 3.2ms preprocess, 305.8ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.])
conf: tensor([0.9747])
data: tensor([[309.6078, 178.4059, 390.5472, 252.7588,   0.9747,   0.0000]])
id: None
is_track: False
orig_shape: (480, 640)
shape: torch.Size([1, 6])
xywh: tensor([[350.0775, 215.5823,  80.9394,  74.3529]])
xywhn: tensor([[0.5470, 0.4491, 0.1265, 0.1549]])
xyxy: tensor([[309.6078, 178.4059, 390.5472, 252.7588]])
xyxyn: tensor([[0.4838, 0.3717, 0.6102, 0.5266]])

# Model Test

In [10]:
# Code took from the ultralytics repository

import cv2
import numpy as np
from ultralytics import YOLOv10

from ultralytics.utils.checks import check_imshow
from ultralytics.utils.plotting import Annotator, colors

from collections import defaultdict

track_history = defaultdict(lambda: [])
model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train2_final/weights/best.pt")
names = model.model.names

video_path = "/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/video_lab_semiopen_1______3.avi"
cap = cv2.VideoCapture(video_path)
assert cap.isOpened(), "Error reading video file"

w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

result = cv2.VideoWriter("video_lab_semiopen_1______3_object_tracking.avi",
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       fps,
                       (w, h))

while cap.isOpened():
    success, frame = cap.read()
    if success:
        results = model.track(frame, persist=True, verbose=False)
        boxes = results[0].boxes.xyxy.cpu()

        if results[0].boxes.id is not None:

            # Extract prediction results
            clss = results[0].boxes.cls.cpu().tolist()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            confs = results[0].boxes.conf.float().cpu().tolist()

            # Annotator Init
            annotator = Annotator(frame, line_width=2)

            for box, cls, track_id in zip(boxes, clss, track_ids):
                annotator.box_label(box, color=colors(int(cls), True), label=names[int(cls)])

                # Store tracking history
                track = track_history[track_id]
                track.append((int((box[0] + box[2]) / 2), int((box[1] + box[3]) / 2)))
                if len(track) > 30:
                    track.pop(0)

                # Plot tracks
                #points = np.array(track, dtype=np.int32).reshape((-1, 1, 2))
                #cv2.circle(frame, (track[-1]), 7, colors(int(cls), True), -1)
                #cv2.polylines(frame, [points], isClosed=False, color=colors(int(cls), True), thickness=2)

        result.write(frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

result.release()
cap.release()
cv2.destroyAllWindows()

In [13]:
from ultralytics import YOLOv10

model = YOLOv10("/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/code/object_detection/YOLOv10/runs/detect/train2_final/weights/best.pt")
model.val(data='/Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/dataset.yaml', batch=128)

Ultralytics YOLOv8.1.34 🚀 Python-3.11.7 torch-2.3.0 CPU (Apple M1 Pro)
YOLOv10m summary (fused): 369 layers, 16451542 parameters, 0 gradients, 63.4 GFLOPs


val: Scanning /Users/alexis/Library/CloudStorage/OneDrive-Balayre&Co/Cranfield/Thesis/thesis-github-repository/data/frames/full_dataset_annotated_YOLO/test/labels.cache... 2208 images, 53 backgrounds, 0 corrupt: 100%|██████████| 2208/2208 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [24:37<00:00, 82.10s/it]


                   all       2208       2165      0.992      0.984      0.995      0.984
Speed: 0.6ms preprocess, 664.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x312953a50>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048,    